In [1]:
from datasets import load_dataset
import markovify

dataset = load_dataset("biglam/gutenberg-poetry-corpus")
dataset['train'] = dataset['train'][:12000]

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
text_model = markovify.Text(dataset['train']['line'])
for i in range(5):
    print(text_model.make_sentence())

Thus SATAN; and him thus the Angelic Vertue answerd milde.
With HALLELUIAHS: Thus was born my Hiawatha,
All its fire was in the troubl'd Skie, and on a Mount, with Pyramids and Towrs
As he howled and hurried meeting here,
The rest is true, they found themselves imprisoned


In [3]:
lines = []
for i in range(5):
    lines.append(text_model.make_sentence())

for line in lines:
    print(line)

Up into the dusk of evening,
Buried in the river,
And from about his Throne, how good, how faire,
Filled the air a moment,
There they stood, and gave to rule,


In [4]:
from textblob import TextBlob
import pandas as pd

def sentiment_analysis(poetry):
    def getSubjectivity(text):
        return TextBlob(text).sentiment.subjectivity

    def getPolarity(text):
        return TextBlob(text).sentiment.polarity

    poetry['TextBlob_Subjectivity'] = poetry['line'].apply(getSubjectivity)
    poetry['TextBlob_Polarity'] = poetry['line'].apply(getPolarity)

    def getAnalysis(score):
        if score < 0:
            return 'Negative'
        elif score == 0:
            return 'Neutral'
        else:
            return 'Positive'

    poetry['TextBlob_Analysis'] = poetry['TextBlob_Polarity'].apply(getAnalysis)
    
    return poetry

# Poem example
poem = {
    'line': lines
}

poetry_df = pd.DataFrame(poem)

result = sentiment_analysis(poetry_df)
print(result)


                                              line  TextBlob_Subjectivity  \
0                     Up into the dusk of evening,                    0.0   
1                             Buried in the river,                    0.0   
2  And from about his Throne, how good, how faire,                    0.6   
3                         Filled the air a moment,                    0.9   
4              There they stood, and gave to rule,                    0.0   

   TextBlob_Polarity TextBlob_Analysis  
0                0.0           Neutral  
1                0.0           Neutral  
2                0.7          Positive  
3                0.4          Positive  
4                0.0           Neutral  


In [5]:
import gensim.downloader

glove_vectors = gensim.downloader.load('glove-wiki-gigaword-300')

def find_synonym(word):
    return glove_vectors.most_similar(word)[0][0]

In [7]:
def replace_with_sinonims(poem):
    new_poem = []
    for line in poem:
        new_vers = ''
        for word in line.split(' '):
            from numpy.random import randint
            p = randint(1,101)
            if p % 2 == 0:
                try:
                    word = find_synonym(word)
                except:
                    word = word
            new_vers += word + ' '
        new_poem.append(new_vers)

    return new_poem

new_poem = replace_with_sinonims(lines)
for line in new_poem:
    print(line)

Up out the dusk the evening, 
Buried in of river, 
And back about he Throne, how good, what faire, 
Filled of air a moment, 
There have stood, and gave to rule, 


In [ ]:
from nltk.translate.bleu_score import sentence_bleu

reference = [[word for word in line] for line in dataset['train']['line']]

candidate = [word for line in lines for word in line ]

score = sentence_bleu(reference, candidate)

print(score)

0.8603600215528934
